# Train
### 1. 데이터 처음 학습하기

1. `python src/KoBART/train.py` 실행


2. 입력값   
    - **train_file**   
    `train 데이터 주소` 입력
    - **test_file**   
    `test 데이터 주소` 입력
    - **mode**  
    `train` 입력
    - **batch_size**   
    `batch size` (ex.16) 입력  
    auto_scale_batch_size='power' 등을 설정하더라도 임의의 값을 입력해야 함
    - **num_workers**  
    `num_workers` (ex.10) 입력   
    가상 cpu 코어 개수 : 72  
    물리적 코어 개수 = 가상 cpu 코어 개수의 절반 = 36   
    적절한 num workers = 물리적 코어 개수의 절반 = 18  
    num workers 18로 진행시 -> Bus error (out of shared memory) -> 도커의 shared memory 변경하면 오류 해결 가능할 것으로 예상
    - **gradient_clip_val**   
    `default 값`인 1.0 입력    
    - **gpus**   
    `제공되는 gpu의 개수` (ex.2) 인 입력
    - **accelerator**   
    multi gpu 사용이므로`ddp` 입력  
    - **max_epochs**  
    `최대 epoch 수` (ex.50) 입력
    - **default_root_dir**  
    `학습 결과를 저장할 폴더 경로` 입력
    
    
3. 저장된 학습 결과 확인  
    **default_root_dir** 폴더 안에 생성되는 폴더 및 파일들
    - **kobart_summary-model_chp**   
        : `epoch=xx-val_loss=x.xxx.ckpt` 의 형태로 체크포인트 파일 생성
    - **tb_logs > default**   
        : `version_x` 의 형태로 버전 폴더 생성   
        : 버전 폴더 안 `events.out.tfevents.xxxxxxxxxxxx`, `hparams.yaml` 의 형태로 이벤트 파일과 하이퍼파라미터 파일 생성
    - **kobart_summary-last.ckpt**   
        : 가장 마지막 체크포인트 파일 생성

In [ ]:
!python src/KoBART/train.py \
--train_file='rsc/Data/Training/AI_hub/한국어대화요약/train_v1.csv' \
--test_file='rsc/Data/Validation/AI_hub/한국어대화요약/dev_v1.csv' \
--mode='train' \
--batch_size=16 \
--num_workers=10 \
--gradient_clip_val=1.0 \
--gpus=2 \
--accelerator=ddp \
--max_epochs=50 \
--default_root_dir='rsc/By_domain_ckpt/aihub한국어대화요약'

INFO:root:Namespace(accelerator='ddp', accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=16, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='rsc/KoBART-AIHub/aihub한국어대화요약2', deterministic=False, distributed_backend=None, enable_pl_optimizer=True, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=2, gradient_clip_val=1.0, hparams_file=None, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=50, max_len=512, max_steps=None, min_epochs=1, min_steps=None, mode='train', model_path=None, move_metrics_to_cpu=False, num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=10, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None, progress_ba

### 2. 데이터 이어 학습하기 (체크포인트 이어 학습)

1. `python src/KoBART/train.py` 실행


2. 입력값
    - **checkpoint_path**    
    학습한 모델의 체크포인트 중 val_loss 가 가장 낮은 `체크포인트 파일 경로` 입력
    - **hparams_file**   
    학습한 모델의 `하이퍼파라미터 파일 경로` 입력
    - 나머지는 위와 동일
    
    
3. 저장된 학습 결과 확인   
    위와 동일 

In [ ]:
!python src/KoBART/train.py \
--train_file='rsc/Data/Training/AI_hub/한국어대화요약/train_v1.csv' \
--test_file='rsc/Data/Validation/AI_hub/한국어대화요약/dev_v1.csv' \
--checkpoint_path='rsc/By_domain_ckpt/dacon신문기사/kobart_summary-model_chp/epoch=02-val_loss=1.326.ckpt' \
--hparams_file='rsc/By_domain_ckpt/dacon신문기사/tb_logs/default/version_1/hparams.yaml' \
--mode='train' \
--batch_size=16 \
--num_workers=10 \
--gradient_clip_val=1.0 \
--gpus=2 \
--accelerator=ddp \
--max_epochs=50 \
--default_root_dir='rsc/By_domain_ckpt/aihub한국어대화요약_dacon신문기사'

# Test
### 학습한 모델로 테스트하기 & rouge 값 출력하기

1. `python src/KoBART/train.py` 실행


2. 입력값   
    - **test_file**   
    `test 데이터 주소` 입력
    - **mode**  
    `test` 입력
    - **checkpoint_path**    
    학습한 모델의 체크포인트 중 val_loss 가 가장 낮은 `체크포인트 파일 경로` 입력
    - **hparams_file**   
    학습한 모델의 `하이퍼파라미터 파일 경로` 입력
    - **default_root_dir**  
    `test 결과를 저장할 폴더 경로` 입력
    - 나머지는 위와 동일
    
    
3. 저장된 테스트 결과 확인  
    **default_root_dir** 폴더 안에 생성되는 폴더 및 파일들
    - **rouge_score.csv**     
        : csv 형태로 rouge score가 출력된 파일 생성
    - **tb_logs > default**   
        : `version_x` 의 형태로 버전 폴더 생성   
        : 버전 폴더 안 `events.out.tfevents.xxxxxxxxxxxx`, `hparams.yaml` 의 형태로 이벤트 파일과 하이퍼파라미터 파일 생성
    - 이 외 파일은 생성되지 않음

In [23]:
!python src/KoBART/train.py \
--train_file='rsc/Data/Training/Dacon/train_v1.csv' \
--test_file='rsc/Data/Validation/Dacon/dev_v1.csv' \
--mode='test' \
--checkpoint_path='rsc/By_hpram_ckpt/batch_4/not_lr_finder/kobart_summary-model_chp/epoch=02-val_loss=1.358.ckpt' \
--hparams_file='rsc/By_hpram_ckpt/batch_4/not_lr_finder/tb_logs/default/version_0/hparams.yaml' \
--batch_size=16 \
--num_workers=10 \
--gpus=2 \
--accelerator=ddp \
--default_root_dir='rsc/By_hpram_ckpt/batch_4/not_lr_finder/test_dacon'

INFO:root:Namespace(accelerator='ddp', accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=16, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path='rsc/KoBART-Dacon/batch_4/not_lr_finder/num_workers4/kobart_summary-model_chp/epoch=02-val_loss=1.358.ckpt', default_root_dir='rsc/KoBART-test', deterministic=False, distributed_backend=None, enable_pl_optimizer=True, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=2, gradient_clip_val=0, hparams_file='rsc/KoBART-Dacon/batch_4/not_lr_finder/num_workers4/tb_logs/default/version_0/hparams.yaml', limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=1000, max_len=512, max_steps=None, min_epochs=1, min_steps=None, mode='test', model_path=None, move_metrics_to_cpu=False, num_nodes=1, num_p

In [26]:
!python src/KoBART/download_binary.py

Download config.json
Downloading...
From: https://drive.google.com/uc?id=1H13loH6dS_2c2Z21kaBtgz42QsjkdAwO
To: /home/tf1/kobart_summary/config.json
100%|██████████████████████████████████████| 1.20k/1.20k [00:00<00:00, 2.95MB/s]
Download pytorch_model.bin
Downloading...
From: https://drive.google.com/uc?id=1D7BAXK_0faWW39c0ptE3FtROsVRbTNwI
To: /home/tf1/kobart_summary/pytorch_model.bin
496MB [00:44, 11.1MB/s] 
